# 看看模型和结构和输入输出

In [46]:
import os
import json
import sys
from pprint import pprint
import numpy as np

# 尝试覆盖掉已经安装的 paddlenlp
sys.path.insert(0, r"G:\code\github\PaddleNLP")

from paddlenlp.datasets import load_dataset
from paddlenlp.prompt import (
    PromptModelForSequenceClassification,
    PromptTrainer,
    PromptTuningArguments,
    UTCTemplate,
)
from paddlenlp.transformers import UTC, AutoTokenizer, export_model

sys.path.append(r"G:\code\github\PaddleNLP\applications\zero_shot_text_classification")
from utils import UTCLoss, read_local_dataset

In [11]:
model_dir = "utc-base"
model_dir = "utc-base"
data_dir = r"G:\dataset\text_classify\baidu_utc_medical\raw"
train_file = "train.txt"
dev_file = "dev.txt"

In [29]:
from paddlenlp.transformers.ernie.tokenizer import ErnieTokenizer

# Load the pretrained language model. 加载模型和分词器
tokenizer: ErnieTokenizer = AutoTokenizer.from_pretrained(model_dir)
model = UTC.from_pretrained(model_dir)

[2023-07-15 09:10:55,207] [    INFO] - We are using (<class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'>, False) to load 'utc-base'.
[2023-07-15 09:10:55,208] [    INFO] - Already cached C:\Users\zhenh\.paddlenlp\models\utc-base\utc_base_vocab.txt
[2023-07-15 09:10:55,223] [    INFO] - tokenizer config file saved in C:\Users\zhenh\.paddlenlp\models\utc-base\tokenizer_config.json
[2023-07-15 09:10:55,224] [    INFO] - Special tokens file saved in C:\Users\zhenh\.paddlenlp\models\utc-base\special_tokens_map.json
[2023-07-15 09:10:56,025] [    INFO] - All model checkpoint weights were used when initializing UTC.

[2023-07-15 09:10:56,025] [    INFO] - All the weights of UTC were initialized from the model checkpoint at utc-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use UTC for predictions without further training.


In [8]:
# Define template for preprocess and verbalizer for postprocess. 看看这个模板是怎么生成的
template = UTCTemplate(tokenizer, 512)

[2023-07-15 08:50:11,392] [    INFO] - Assigning ['[O-MASK]'] to the additional_special_tokens key of the tokenizer


In [12]:
# Load and preprocess dataset. 加载数据集
train_ds = load_dataset(
    read_local_dataset,
    data_path=data_dir,
    data_file=train_file,
    lazy=False,
)
dev_ds = load_dataset(
    read_local_dataset,
    data_path=data_dir,
    data_file=dev_file,
    lazy=False,
)

[2023-07-15 08:52:29,140] [ WARNING] - Skip 0 examples.
[2023-07-15 08:52:29,141] [ WARNING] - Skip 0 examples.


In [19]:
print(len(train_ds))
pprint(train_ds[0])
for label, index in zip(train_ds[0]["choices"], train_ds[0]["labels"]):
    if index == 1:
        print(label)

45
{'choices': ['病情诊断',
             '治疗方案',
             '病因分析',
             '指标解读',
             '就医建议',
             '疾病表述',
             '后果表述',
             '注意事项',
             '功效作用',
             '医疗费用',
             '其他'],
 'labels': [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'question': '',
 'text_a': '糖尿病蜜月期有永久性的吗',
 'text_b': ''}
疾病表述


In [20]:
# Define the criterion. 定义损失
criterion = UTCLoss()

# Initialize the prompt model. 初始化提示学习的模型, 没有用到第三个参数 verbalizer
prompt_model = PromptModelForSequenceClassification(
    model, template, None, freeze_plm=False, freeze_dropout=False
)

# 训练时的输入字段

In [47]:
# 训练时的输入字段
for key, val in template(train_ds[0]).items():
    print(key, type(val), val)

    if key == "input_ids":
        print(len(val))
        print(tokenizer.convert_ids_to_tokens(val))
        print(tokenizer.decode(val))
        input_ids = val
    if key == "attention_mask":
        attention_mask = val
    print()

soft_token_ids <class 'list'> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

input_ids <class 'list'> [1, 39980, 396, 182, 1395, 533, 39980, 301, 641, 58, 487, 39980, 396, 196, 59, 764, 39980, 288, 275, 273, 555, 39980, 113, 386, 81, 454, 39980, 1095, 396, 197, 779, 39980, 49, 228, 197, 779, 39980, 490, 221, 104, 236, 39980, 369, 344, 25, 29, 39980, 386, 641, 453, 29, 39980, 63, 44, 2, 1237, 1601, 396, 2027, 136, 195, 9, 885, 876, 92, 5, 1114, 2, 2]
69
['[CLS]', '[O-MASK]', '病', '情', '诊', '断', '[O-MASK]', '治', '疗', '方', '案', '[O-MASK]', '病', '因', '分', '析', '[O-MASK]', '指', '标', '解', '读', '[O-MASK]', '就', '医', '建', '议', '[O-MASK]', '疾', '病', '表', '述', '[O-MASK]', '后', '果', '表', '述', '[O-MASK]', '注', '意', '事', '项', '[O-MASK]', '功', '效', '作', '用', '[O-MASK]', '医', '疗', '费', '用', '[O-MASK]', '其', '他', '[SEP]', '糖', '尿', '病', '蜜', 

In [45]:
for index in range(attention_mask.shape[0]):
    print(attention_mask[index].tolist())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, -10000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -10000.0, -10000.0, -10000.0, -10000.

In [62]:
"""
创建注意力掩码
"""
omask_token = "[O-MASK]"
omask_id = tokenizer.convert_tokens_to_ids(omask_token)
input_ids = np.array(input_ids)
# 这是个矩阵, shape 是 [len(input_ids), len(input_ids)]
attention_mask = np.ones([len(input_ids), len(input_ids)])
# 都取 [0] 是因为 where 会返回一个元组
omask_index = np.where(input_ids == omask_id)[0].tolist()
cls_indices = np.where(input_ids == tokenizer.cls_token_id)[0]
sep_indices = np.where(input_ids == tokenizer.sep_token_id)[0]

print("omask_index", omask_index)
print("cls_indices", cls_indices)
print("sep_indices", sep_indices)

# 找一个最小的且在 omask_index 之后的 cls_index 和 sep_index
cls_index = len(input_ids)
for idx in cls_indices:
    if idx > omask_index[-1]:
        cls_index = idx
        break
print("cls_index", cls_index)
sep_index = len(input_ids)
for idx in sep_indices:
    if idx > omask_index[-1]:
        sep_index = idx
        break
print("sep_index", sep_index)
# opt 的开始和结束位置
opt_begin = omask_index[0]
opt_end = min(cls_index, sep_index)
print(opt_begin, opt_end)
# 整一段都要忽略
attention_mask[opt_begin:opt_end, opt_begin:opt_end] = 0
omask_index.append(opt_end)
for opt_begin, opt_end in zip(omask_index[:-1], omask_index[1:]):
    print(opt_begin, opt_end)
    # 其中是选项文本的部分要保留
    attention_mask[opt_begin:opt_end, opt_begin:opt_end] = 1
# 这个就是将 attention_mask 中的 0 变成 -1e4, 1 变成 0
attention_mask = (attention_mask - 1) * 1e4

omask_index [1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51]
cls_indices [0]
sep_indices [54 67 68]
cls_index 69
sep_index 54
1 54
1 6
6 11
11 16
16 21
21 26
26 31
31 36
36 41
41 46
46 51
51 54


# 模型架构

In [64]:
template

UTCTemplate()

In [66]:
# 用的 utc-base 是 12 层的
model

UTC(
  (ernie): ErnieModel(
    (embeddings): ErnieEmbeddings(
      (word_embeddings): Embedding(39981, 768, padding_idx=0, sparse=False)
      (position_embeddings): Embedding(2048, 768, sparse=False)
      (token_type_embeddings): Embedding(4, 768, sparse=False)
      (layer_norm): LayerNorm(normalized_shape=[768], epsilon=1e-12)
      (dropout): Dropout(p=0.1, axis=None, mode=upscale_in_train)
    )
    (encoder): TransformerEncoder(
      (layers): LayerList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiHeadAttention(
            (q_proj): Linear(in_features=768, out_features=768, dtype=float32)
            (k_proj): Linear(in_features=768, out_features=768, dtype=float32)
            (v_proj): Linear(in_features=768, out_features=768, dtype=float32)
            (out_proj): Linear(in_features=768, out_features=768, dtype=float32)
          )
          (linear1): Linear(in_features=768, out_features=3072, dtype=float32)
          (dropout): Dropout(p=0, axis=No

In [68]:
# 组合了 UTC 模型和 template
prompt_model

PromptModelForSequenceClassification(
  (plm): UTC(
    (ernie): ErnieModel(
      (embeddings): ErnieEmbeddings(
        (word_embeddings): Embedding(39981, 768, padding_idx=0, sparse=False)
        (position_embeddings): Embedding(2048, 768, sparse=False)
        (token_type_embeddings): Embedding(4, 768, sparse=False)
        (layer_norm): LayerNorm(normalized_shape=[768], epsilon=1e-12)
        (dropout): Dropout(p=0.1, axis=None, mode=upscale_in_train)
      )
      (encoder): TransformerEncoder(
        (layers): LayerList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiHeadAttention(
              (q_proj): Linear(in_features=768, out_features=768, dtype=float32)
              (k_proj): Linear(in_features=768, out_features=768, dtype=float32)
              (v_proj): Linear(in_features=768, out_features=768, dtype=float32)
              (out_proj): Linear(in_features=768, out_features=768, dtype=float32)
            )
            (linear1): Linear(in_featur

# 预测

如果使用 fp16, 需要安装这些 onnx 相关的依赖

```
onnx                     1.14.0
onnxconverter-common     1.13.0
onnxruntime-gpu          1.15.1
paddle2onnx              1.0.6
```

In [100]:
from paddlenlp import Taskflow

schema = ["病情诊断", "治疗方案", "病因分析", "指标解读", "就医建议", "疾病表述", "后果表述", "注意事项", "功效作用", "医疗费用", "其他"]
my_cls = Taskflow("zero_shot_text_classification", model="utc-base", schema=schema, task_path=r"G:\code\github\PaddleNLP\outputs\plm", precision="fp32")
task_instance = my_cls.task_instance
pprint(my_cls("月经期间刮痧拔罐会引起身体什么"))

[2023-07-16 13:46:13,850] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'G:\code\github\PaddleNLP\outputs\plm'.
[2023-07-16 13:46:13,876] [    INFO] - Assigning ['[O-MASK]'] to the additional_special_tokens key of the tokenizer


[{'predictions': [{'label': '后果表述', 'score': 0.9999968636902635}],
  'text_a': '月经期间刮痧拔罐会引起身体什么'}]


In [91]:
!pip list | findstr onnx

onnx                     1.14.0
onnxconverter-common     1.13.0
onnxruntime-gpu          1.15.1
paddle2onnx              1.0.6


In [102]:
task_instance._custom_model, task_instance.is_static_model, task_instance._predictor_type, task_instance._num_threads

(True, False, 'paddle-inference', 12)

In [103]:
type(task_instance.predictor)

paddle.fluid.libpaddle.PaddleInferPredictor